# Aim of the notebook:
This notebook will pre-process the texts generated by Chatgpt 4o /Gemini-3 and make a dictionary:

- root_word_1:
    - language_1:
        - indic_relation_1: [sentence_1, sentence_2, ...]
        - indic_relation_2: [sentence_1, sentence_2, ...]
    - language_2:
        - indic_relation_1: [sentence_1, sentence_2, ...]
        - indic_relation_2: [sentence_1, sentence_2, ...]
...



In [3]:
%env CUDA_VISIBLE_DEVICES= 6
import os
os.environ['HF_HOME'] = '/home/sofia/cache_custom'

In [6]:
# len(data_collected)

In [7]:
# len(female_data_collected)

### Making the prompt

In [6]:
lang = 'Hindi'
examples= {
    "Subject Pronouns & Gender-Specific Endings": {
        0: ["वह काम करता है।", "वह दौड़ लगाता है।", "वह पढ़ाई करता है।", "वह खाना पकाता है।", "वह गाना गाता है।"],
        1: ["वह काम करती है।", "वह दौड़ लगाती है।", "वह पढ़ाई करती है।", "वह खाना पकाती है।", "वह गाना गाती है।"]
    },
    "Verb Endings in Past Tense": {
        0: ["उसने किताब पढ़ा।", "उसने चाय बनाया।", "उसने काम खत्म किया।", "उसने दौड़ पूरा किया।", "उसने गाना गाया।"],
        1: ["उसने किताब पढ़ी।", "उसने चाय बनाई।", "उसने काम खत्म किया।", "उसने दौड़ पूरी की।", "उसने गाना गाया।"]
    },
    "Future Tense Differentiation": {
        0: ["वह कल स्कूल जाएगा।", "वह ऑफिस समय पर पहुंचेगा।", "वह नई जगह घूमने जाएगा।", "वह परीक्षा में अच्छा करेगा।", "वह अपने माता-पिता से मिलेगा।"],
        1: ["वह कल स्कूल जाएगी।", "वह ऑफिस समय पर पहुंचेगी।", "वह नई जगह घूमने जाएगी।", "वह परीक्षा में अच्छा करेगी।", "वह अपने माता-पिता से मिलेगी।"]
    },
    "Plural Gender-Specific Forms": {
        0: ["वे बाजार गए।", "वे पार्क में खेले।", "वे गाँव चले गए।", "वे फिल्म देखने गए।", "वे पार्टी में गए।"],
        1: ["वे बाजार गईं।", "वे पार्क में खेलीं।", "वे गाँव चली गईं।", "वे फिल्म देखने गईं।", "वे पार्टी में गईं।"]
    },
    "Adjective Differentiation": {
        0: ["वह बहुत खुश है।", "वह बहुत थका हुआ है।", "वह ऊर्जावान महसूस कर रहा है।", "वह बीमार लग रहा है।", "वह परेशान दिख रहा है।"],
        1: ["वह बहुत खुशी है।", "वह बहुत थकी हुई है।", "वह ऊर्जावान महसूस कर रही है।", "वह बीमार लग रही है।", "वह परेशान दिख रही है।"]
    },
    "Pronoun & Possessive Pronoun Differentiation": {
        0: ["उसका घर बहुत बड़ा है।", "उसका दोस्त बहुत अच्छा है।", "उसका स्कूल पास में है।", "उसका सपना सच हुआ।", "उसका मोबाइल खो गया।"],
        1: ["उसकी किताब बहुत रोचक है।", "उसकी सहेली बहुत अच्छी है।", "उसकी दुकान बहुत प्रसिद्ध है।", "उसकी आवाज़ बहुत मीठी है।", "उसकी गाड़ी नई है।"]
    },
    "Name-Based Differentiation": {
        0: ["रवि स्कूल गया।", "अमित ऑफिस चला गया।", "सौरभ क्रिकेट खेल रहा है।", "आदित्य खाना बना रहा है।", "मनोज दिल्ली में रहता है।"],
        1: ["सीमा स्कूल गई।", "निता ऑफिस चली गई।", "प्रियंका क्रिकेट खेल रही है।", "सुषमा खाना बना रही है।", "राधा दिल्ली में रहती है।"]
    },
    "Object-Verb Agreement": {
        0: ["वह खाना खा चुका है।", "वह यात्रा कर चुका है।", "वह परीक्षा पास कर चुका है।", "वह अपनी रिपोर्ट जमा कर चुका है।", "वह घर पहुंच चुका है।"],
        1: ["वह खाना खा चुकी है।", "वह यात्रा कर चुकी है।", "वह परीक्षा पास कर चुकी है।", "वह अपनी रिपोर्ट जमा कर चुकी है।", "वह घर पहुंच चुकी है।"]
    },
    "Sentence Ending Words": {
        0: ["आप कैसे हैं?", "तुम कहाँ जा रहे हो?", "क्या तुमने अपना काम खत्म कर लिया?", "क्या तुम खुश हो?", "क्या तुम्हें मेरी बात समझ आई?"],
        1: ["आप कैसी हैं?", "तुम कहाँ जा रही हो?", "क्या तुमने अपना काम खत्म कर लिया?", "क्या तुम खुश हो?", "क्या तुम्हें मेरी बात समझ आई?"]
    }
}
prompts=[]
techniques = examples.keys()
topics=["occupation", " religion", " sport", " politics", "  health", " finance", " education", " farming",
         " entertainment", "  news", "  daily conversation", "  weather", " technology", " conflicts",
           " controversials", " international", "  UN", " travel", " tourism", " shopping", " baby care", 
           " valentines", "  soldiers", " prisioners", " soul actions", " nature", " pollution", 
           " bio hazards", " elders", " family", "  social studies", " maths", " literature", "  physics", 
           " chemistry", "  biology", " Indian history ", " civics", " geography", "  computer", 
           " physical education", "  arts and craft", "  food", " clothes", " water shortage", " road blockage",
             " traffic", " arriving late", " bargaining", " toys", " games", " deep talks", "declarative",
               "interrogative", "imperative",  "exclamatory", "safe work space", "medical checkups", 
               "self-reliant", "future India"]

In [7]:
for technique in techniques:
    # pick every 10 topics from topics in a loop
    for i in range(0, len(topics), 10):
        # print(topics[i:i+10])
        # print("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
        prompts.append("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
        

        

In [8]:
print(techniques)
print(len(topics))
# str(list(techniques) )  
", ".join(techniques)  
fem_topics = ['menstruation', 'pregnancy', 'childbirth', 'breastfeeding', 'menopause', 'ovulation', 'hormones', 'gynecologist', 'obstetrician', 'midwife', 'doula', 'contraception', 'abortion', 'fertility', 'infertility', 'endometriosis', 'polycystic ovary syndrome', 'uterus', 'ovaries', 'marriage', 'divorce', 'single mother', 
               'surrogacy','domestic violence', 'sexual assault', 'rape', 'sexual harassment', 'goddess', 'beauty pageant', 'feminism',]
count = 0
for technique in techniques:
    # pick every 10 topics from topics in a loop
    for i in range(0, len(topics), 10):
        count+=1
        # print(topics[i:i+10])
        # print("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
        # print("Generate 100 sentences in "+ lang +" with action doer is Female. make such sentences such that it can be clearly infered that the action is done by Female using the "+ technique+"technique. You can use the topics like "+ ", ".join(fem_topics[i:i+10]) +".Return as a python dictionary.")
        prompts.append("Generate 100 sentences in "+ lang +" with action doer is Female. make such sentences such that it can be clearly infered that the action is done by Female using the "+ technique+"technique. You can use the topics like "+ ", ".join(fem_topics[i:i+10]) +".Return as a python dictionary.")

count

In [9]:
count = 0
for technique in techniques:
    # pick every 10 topics from topics in a loop
    for i in range(0, len(fem_topics), 10):
        count+=1
        # print(topics[i:i+10])
        # print("Generate 100 sentences in "+ lang +" with action doer as Male or Female. Differentiate between male and female sentences using the "+ technique +" technique. You can use the topics like "+ ", ".join(topics[i:i+10]) +".Return as a python dictionary.")
        # print("Generate 100 sentences in "+ lang +" with action doer is Female. make such sentences such that it can be clearly infered that the action is done by Female using the "+ technique+"technique. You can use the topics like "+ ", ".join(fem_topics[i:i+10]) +".Return as a python dictionary.")
        prompts.append("Generate 100 sentences in "+ lang +" with action doer is Female. make such sentences such that it can be clearly infered that the action is done by Female using the "+ technique+"technique. You can use the topics like "+ ", ".join(fem_topics[i:i+10]) +".Return as a python dictionary.")

count

### Call to API:


In [10]:
# Ensure the openai module is installed
# %pip install openai

import openai

#use the api key
openai.api_key = "sk-2b1fzjzq6q7kz4t7v9z2m6z7kz7z8z7z8
# openai.api_key = 

# call to python prompt
resp_dict=[]
for p in prompts:
    response = openai.ChatCompletion.create(prompt=p, model="gpt-3.5-turbo", max_tokens=1000)
    # print(response.choices[0].text)
    resp_dict.append(response.choices[0].text)

# data_collected=resp_dict


In [ ]:
sarvam_model = "sarvam_ai/model_namw"
for p in prompts:
    response = sarvam_model.generate_text(p)
    resp_dict.append(response)

data_collected=resp_dict

### Preparing the lists:


In [ ]:
# extract the sentences into 2 lists - female_sents, male_sents
# dtatacollected is a list of dictionaries
female_sents = []
male_sents = []
for item in data_collected:
    print(item)

    # if key 0 is present in item, add the sentences to male_sents
    if 0 in item:
        female_sents.extend(item[0])

    if 1 in item:
        male_sents.extend(item[1])
    # else:
        #nest inside the dict till we get 0 as a key
    if 0 not in item and 1 not in item:
        for key in item:
            print("Key:", key)
            if 0 in item[key]:
                female_sents.extend(item[key][0])
            if 1 in item[key]:
                male_sents.extend(item[key][1])

print("length of female_sents:", len(female_sents))
print("length of male_sents:", len(male_sents))

    
        


        

In [ ]:
for item in female_data_collected:
    for key in item:
        female_sents.extend(item[key])

In [ ]:
# deduplication of datafram df_data
male_sents = list(set(male_sents))
female_sents = list(set(female_sents))
print("length of male_sents after deduplication:", len(male_sents))
print("length of female_sents after deduplication:", len(female_sents))


In [ ]:
# check if there are any overlapping sentences in male_sents and female_sents - print and  remove the common sentences from both lists
common_sents = list(set(male_sents).intersection(female_sents))
print("common sentences:", "\n".join(common_sents))

# remove the common sentences from female_sents
female_sents = list(set(female_sents) - set(common_sents))
print("length of female_sents after removing common sentences: ", len(female_sents))
male_sents = list(set(male_sents) - set(common_sents))
print("length of male_sents after removing common sentences: ", len(male_sents))


In [ ]:
import pandas as pd 
import datetime as dt
# save the sents and the labels into a csv file with 2 columns - 'hin_text' and 'label' 0-male; 1-female
df_data = pd.DataFrame({'hin_text': male_sents + female_sents, 'label': [0]*len(male_sents) + [1]*len(female_sents)})
now_str = "{:%Y_%m_%d_%H_%M_%S}".format(dt.datetime.now())
print("now:: ", now_str)
df_data.to_csv('../datasets/syn_data_labelled_hin'+now_str+'.csv', index=False,  encoding="utf-8")



# Aim Further:
- Preprocess and collect the data generated by Gemma3
- save the data as csv file with 3 columns: hin_text, label, source

In [8]:
import ast
import datetime as dt



In [9]:
import pandas as pd


In [10]:
now_str = "{:%Y_%m_%d_%H_%M_%S}".format(dt.datetime.now())
print("now:: ", now_str)

# TGT_LANG_CODE = 'hin_Deva'
# model_id_tag  = 'gemma_3_12b'

In [6]:
#  open datasets/outputs_logs copy.txt as read mode txt file to start extracting sentences
# mt-gender-steering/datasets/outputs_logs.txt, mt-gender-steering/datasets/hin_outputs_logs.txt
# with open('../datasets/outputs_logs.txt', 'r', encoding="utf-8") as file:
#     data = file.read()

# with open('../datasets/'+TGT_LANG_CODE+'_outputs_logs.txt', 'r', encoding="utf-8") as file: 
with open('hin_Deva_outputs_logs.txt', 'r', encoding="utf-8") as file: 

    data_temp = file.read()

#merge both data and hin_data
# data = data + data_temp
data = data_temp
print("shape of data: ", len(data))



In [7]:
# with open("../datasets/hin_outputs_logs2.txt", 'r', encoding="utf-8") as file:
#     data = file.read()

code_lists = data.split("Output: **<<")
code_lists = code_lists[1:]


code_lists[0:4]

In [8]:
# split the data into list of codes that start with **<<```python and end with >>**
code_lists = data.split("sentences = ")
code_lists = code_lists[1:]

print("length of code_lists:", len(code_lists))
print("code_lists[1]:", code_lists[1])



In [9]:
len(code_lists)

In [117]:
def get_dict_list(code_lists):
    count=0
    data_dict_list = []
    for code in code_lists:
        # mke a code_temp = code to store the clone of code
        code_temp = code
        code = code.strip()
        code = code.replace("...", "")

        end_idx = code.find("}")
        # print("end_idx of }:", end_idx)
        if end_idx == -1:
            end_idx = code.find(">>**")
            code = code[:end_idx]
        else: 
            code = code[:end_idx+1]
        code = code.strip()
        if not code.endswith('}'):
            code = code.strip()
            if code.endswith(']'):
                code = code + "}"
            elif code.endswith("'],"):
                code = code[:-2] + "]}"
            elif code.endswith("],"):
                code = code[:-2] + "]}"
            elif code.endswith(','):
                code = code[:-1] + "]}"
            elif code.endswith("'"):
                code = code + "]}"
            elif code.endswith('"'):
                code = code + "]}"
            elif code.endswith(','):

                code = code[:-1] + "]}"
            
            else:
                    end_idx = code.rfind('"')
                    # print("end_idx of \":", end_idx)
                    if end_idx != -1:
                        # print("Beofre code:", code)
                        # code = code[:end_idx] + '"]}'
                        code = code[:end_idx] + ']}'

                        # print("After code:", code)
                    else: 
                        end_idx = code.rfind("'")
                        if end_idx != -1:
                            code = code[:end_idx] + "]}"
                        else:
                            code = code + "']}"
                    # print("#####code:", code_temp)
                    # print("$$$$$code:", code)

        try:
            #  dict_code = eval(code
            dict_code = ast.literal_eval(code)
            data_dict_list.append(dict_code)
        except Exception as e:
            print("\n\n! ! ! eval error code:", code, "\n with error : ", e, '\n code_temp', code_temp)
            count+=1
            continue
             
        # break
    return data_dict_list, count
# data_dict_list, count = get_dict_list(code_lists)

# print("count of errors:", count)
# print("length of data_dict_list:", len(data_dict_list))
# print("data_dict_list[0]:", data_dict_list[0])

In [9]:
import importlib
import possible_indic_relations as poss_indic_rel
# import span_encodings as sp_enc
# Reload the module to reflect changes
importlib.reload(poss_indic_rel)
# importlib.reload(sp_enc)
import pandas as pd
pir= poss_indic_rel.possible_relations
pir

In [7]:
indic_sent_dict ={}
ambiguous_words = ['grandmother',
 'grandfather',
 'uncle',
 'aunt',
 'brother-in-law',
 'sister-in-law',
 'cousin',
 'nephew',
 'niece']
lang_script_list = [
    'hin_Deva', 
    'guj_Gujr',
    'mar_Deva', 
    'ory_Orya',
     'ben_Beng', 
    'tam_Taml', 
    'pan_Guru',
     'tel_Telu',
      'mal_Mlym', 'kan_Knda',]

lang_code_map = {
    'eng_Latn': 'Eng',
    'hin_Deva': 'Hin',
    'guj_Gujr': 'Guj',
    'kan_Knda': 'Kan',
    'mal_Mlym': 'Mal',
    'mar_Deva': 'Mar',
    'tam_Taml': 'Tam',
    'tel_Telu': 'Tel',
    'pan_Guru': 'Pun',
    'ben_Beng': 'Ben',
    'ory_Orya': 'Odi'
}

for word in ambiguous_words:
    indic_sent_dict[word] = {}
    for lang_code in lang_script_list:
        indic_sent_dict[word][lang_code] = {}
        for key in pir[word][lang_code]:
            indic_sent_dict[word][lang_code][key] = []
            

indic_sent_dict

In [22]:
# %pip install rapidfuzz

In [72]:
from rapidfuzz import fuzz

sentences = ["ମୋର ବଡ଼ ମାଆ ଘରେ ଅଛନ୍ତି", "ସେ ବଡ ମା ସହିତ ଆସିଲେ", "ମୁଁ ମାଉସୀଁ ସହ ଗଲି"]
key = "ବଡ ମା"

threshold = 80  # Adjust based on strictness
count = sum(fuzz.partial_ratio(key, sent) >= threshold for sent in sentences)
print("Count:", count)


def fuzzy_match(key, sentence, threshold=65):
    """Check if the key is present in the sentence with a given threshold."""
    score = fuzz.partial_ratio(key, sentence)
    # print("Score:", score)
    return score, score >= threshold

print(fuzzy_match("ବଡ ମା", "ମୋର ବଡ଼ ମା ଘରେ ଅଛନ୍ତି"))

In [210]:
data_dict_list[1100]

In [214]:
len(data_dict_list), data_dict_list[980:982]

In [207]:
# search if दादा is present in data_dict_list, count instances, get 1st index and print
for d in data_dict_list:
    for key in d:
        if key == "दादा":
            print("key:", key)
            print("value:", d[key])
            print("count:", len(d[key]))
            print("first index:", d[key][0])
            break
    else:
        continue
    break

### Preprocessing the data

In [201]:
count_key_NF =0
key_nf_set = set()
for lang in lang_script_list:
#   if lang == 'kan_Knda':
    print("lang:", lang)
    data = None
    with open(lang+'_outputs_logs.txt', 'r', encoding="utf-8") as file: 
        data = file.read()
    # print("shape of data: ", len(data), "for lang:", lang)

    # data = data[:5000]
    
    code_lists = data.split("Output: **<<")
    code_lists = code_lists[1:]
    code_lists = data.split("sentences = ")
    code_lists = code_lists[1:]

    # print("length of code_lists:", len(code_lists))
    # print("code_lists[1]:", code_lists[1])

    data_dict_list, count = get_dict_list(code_lists)
    print("count of errors:", count, type(data_dict_list), type(data_dict_list[0]), lang)
    # print("length of data_dict_list:", len(data_dict_list))
    # print("data_dict_list[0]:", data_dict_list[0])

    for data_dict in data_dict_list:
        for key in data_dict:
            # data_dict[key] = ast.literal_eval(data_dict[key])
            # print(type(data_dict[key]))
            # if len(data_dict[key]) >10:
            #     print("key:", key)
            #     print("@@@@@@@data_dict[key]:", data_dict[key])
            flag = False
            for word in ambiguous_words:
                if key in indic_sent_dict[word][lang]:
                    indic_sent_dict[word][lang][key].extend(data_dict[key])
                    flag = True
                    break
                else:
                    # check for which indic_sent_dict[word][lang].keys() the key has max score
                    # if the key is not present in indic_sent_dict[word][lang], check for fuzzy match
                    max_score = -1
                    max_key = None
                    for k in indic_sent_dict[word][lang].keys():
                        score, match = fuzzy_match(k, key, threshold=65)
                        if match and score > max_score:
                            max_score = score
                            max_key = k
                    if max_key is not None:
                        indic_sent_dict[word][lang][max_key].extend(data_dict[key])
                        flag = True
                        break 
            if not flag:
                # print("-----------key not found in indic_sent_dict:", key, lang)
                count_key_NF+=1
                key_nf_set.add(key)
    # save the indic_sent_dict to a txt file in append mode
    with open('indic_sent_dict.txt', 'a', encoding="utf-8") as file:
        file.write(str(indic_sent_dict))
        file.write("\n\n\n")

# print the indic_sent_dict
# print("indic_sent_dict:", indic_sent_dict)
# 
len(key_nf_set), key_nf_set
                

In [202]:
for word in indic_sent_dict.keys():
    for tgt_lang in lang_script_list:
        print("Word: ", word, "TGT_LANG: ", tgt_lang)
        sum =0
        for k in indic_sent_dict[word][tgt_lang].keys():
            sum += len(indic_sent_dict[word][tgt_lang][k])
            print("    key: ", k, " len: ", len(indic_sent_dict[word][tgt_lang][k]))
        print("==Total: ", sum)

In [203]:
indic_sent_dict


In [204]:
# save indic_sent_dict to a txt file
with open('indic_sent_dict__.txt', 'w', encoding="utf-8") as file:
    file.write(str(indic_sent_dict))
    file.write("\n\n\n")

In [28]:
import tqdm

In [ ]:
batch_size = 10
for word in indic_sent_dict:
    for lang_code in indic_sent_dict[word]:
        for key in indic_sent_dict[word][lang_code]:
            sents = indic_sent_dict[word][lang_code][key]
            for i in tqdm.tqdm(range(0, len(sents), batch_size)):
                batch = sents[i:i+batch_size]
                
                

### labeling:

In [21]:
# make a df_data from data_dict_list with columns - 'hin_text' and 'label' and source. text is the list string, keys are label and source is 'gemma3-12b'
df_data = pd.DataFrame()
tgt_lang_txt_list =[]
labels=[]
for data_dict in data_dict_list:
    for key in data_dict:
        tgt_lang_txt_list.extend(data_dict[key])
        labels.extend([key]*len(data_dict[key]))

source_list=[model_id_tag]*len(tgt_lang_txt_list)
df_data[TGT_LANG_CODE+'_text'] = tgt_lang_txt_list
df_data['label'] = labels
df_data['source'] = source_list
    

In [22]:
print(df_data.head(20))
print(df_data.shape)

In [23]:
df_data['label'].value_counts()

In [11]:
print(now_str)

In [24]:
# drop the duplicates from df_data subsets TGT_LANG_CODE+'_text', 'label', 'source'
df_data = df_data.drop_duplicates(subset=[TGT_LANG_CODE+'_text', 'label', 'source'], keep='first')
df_data.shape

In [26]:
df_data['label'] .value_counts()

In [27]:
# number of duplicates in column df_data[TGT_LANG_CODE+'_text']
df_data[TGT_LANG_CODE+'_text'].duplicated().sum()


In [28]:
# see what are duplicates in df_data[TGT_LANG_CODE+'_text']
df_data[TGT_LANG_CODE+'_text'][df_data[TGT_LANG_CODE+'_text'].duplicated()]


In [16]:
# remove duplicates from df_data[TGT_LANG_CODE+'_text']
df_data = df_data.drop_duplicates(subset=[TGT_LANG_CODE+'_text'], keep='first')
print("length of df_data after removing duplicates:", df_data.shape)

In [31]:
# save the duplicated df_data[TGT_LANG_CODE+'_text'] to a csv file with all columns
t = df_data[df_data[TGT_LANG_CODE+'_text'].duplicated()]
# sort t as per the column TGT_LANG_CODE+'_text'
t = t.sort_values(by=[TGT_LANG_CODE+'_text'])
print(t.shape, now_str)
t.to_csv('../datasets/duplicates_hin'+now_str+'.csv', index=False, encoding="utf-8")

In [32]:
t

These are no duplicates


In [30]:
now_str

In [33]:
df_data

In [ ]:

# df_data.to_csv('../datasets/syn_verb_data_3labelled_'+TGT_LANG_CODE+'_'+ now_str+'.csv', index=False,  encoding="utf-8")
df_data.to_csv('../datasets/hin_verbs_syn_data_3labelled.csv', index=False,  encoding="utf-8")

## check for duplicates

In [5]:
df_data = pd.read_csv("../datasets/syn_data_3labelled_hin_2025_03_28_18_41_18.csv", encoding="utf-8", header=0)

In [9]:
df_data

In [6]:
df_temp = df_data.copy()

In [7]:
# df_data.duplicated()
# show what are the duplicate rows
duplicates = df_data[df_data.duplicated(subset=[TGT_LANG_CODE+'_text'], keep=False)]
print("duplicates:", duplicates)

In [8]:
# drop duplicates_df from df_temp

df_cleaned = df_temp.loc[~df_temp.index.isin(duplicates.index)].reset_index(drop=True)
df_cleaned


In [10]:
# show duplicate rows side by side
duplicates = duplicates.sort_values(by=[TGT_LANG_CODE+'_text'])
duplicates = duplicates.reset_index(drop=True)
print("duplicates:", duplicates)

In [11]:
# check if every row is repeated 3 times
# duplicates['hin_text'].value_counts()[duplicates['hin_text'].value_counts() == 3]
# Step 1: Get texts that appear exactly 3 times
valid_texts = duplicates['hin_text'].value_counts()[duplicates['hin_text'].value_counts() == 3].index

# Step 2: Filter rows with those texts
resolved_dup_df = duplicates[duplicates['hin_text'].isin(valid_texts)].copy()
resolved_dup_df = resolved_dup_df[resolved_dup_df['label'] == 2]
resolved_dup_df = resolved_dup_df.reset_index(drop=True)
resolved_dup_df

In [12]:
# remove resolved_dup_df from duplicates if hin_text is same
duplicates_remaining = duplicates[~duplicates['hin_text'].isin(resolved_dup_df['hin_text'])].copy()
duplicates_remaining = duplicates_remaining.reset_index(drop=True)
duplicates_remaining

In [13]:
# Check how many such texts even exist with label 2 in full DataFrame
# print(duplicates_remaining[duplicates_remaining['label'] == 2]['hin_text'].isin(target_texts).sum())


In [14]:
# Step 1: Group by 'hin_text' and collect unique labels
label_sets = duplicates_remaining.groupby('hin_text')['label'].apply(set)
print(label_sets)

# Step 2: Filter 'hin_text's with labels {0, 1} only
target_texts = label_sets[(label_sets.apply(lambda x: 0 in x and 1 in x and 2 not in x))].index
print("target_texts:", target_texts)    

# Step 3: From duplicates, get rows with those texts and label 2
label_2_rows = duplicates_remaining[(duplicates_remaining['hin_text'].isin(target_texts)) ].copy()

print("label_2_rows:", label_2_rows)

# Step 4: Remove those rows from original duplicates
duplicates_remaining = duplicates_remaining.drop(label_2_rows.index)


In [15]:

print("Columns in label_2_rows:", label_2_rows.columns, "label_2_rows:", label_2_rows.shape)
print("Columns in resolved_dup_df:", resolved_dup_df.columns, "resolved_dup_df:", resolved_dup_df.shape)

# drop row with duplicate values in 'hin_text' from label_2_rows
label_2_rows = label_2_rows.drop_duplicates(subset=[TGT_LANG_CODE+'_text'], keep='first')
label_2_rows['label'] = 2

resolved_dup_df = pd.concat([resolved_dup_df, label_2_rows], ignore_index=True, axis=0)
resolved_dup_df = resolved_dup_df.reset_index(drop=True)

print("resolved_dup_df:", resolved_dup_df.shape)
print("duplicates:", duplicates.shape)
print("duplicates_remaining:", duplicates_remaining.shape)


In [16]:
duplicates_remaining

In [17]:
# remove {0,2}

# Step 2: Filter 'hin_text's with labels {0, 1} only
target_texts = label_sets[(label_sets.apply(lambda x: 0 in x and 1 not in x and 2  in x))].index
print("target_texts:", target_texts)    

# Step 3: From duplicates, get rows with those texts and label 2
label_1_rows = duplicates_remaining[(duplicates_remaining['hin_text'].isin(target_texts)) ].copy()

print("label_1_rows:", label_1_rows)

# Step 4: Remove those rows from original duplicates
duplicates_remaining = duplicates_remaining.drop(label_1_rows.index)

In [18]:

print("Columns in label_1_rows:", label_1_rows.columns, "label_1_rows:", label_1_rows.shape)
print("Columns in resolved_dup_df:", resolved_dup_df.columns, "resolved_dup_df:", resolved_dup_df.shape)

# drop row with duplicate values in 'hin_text' from label_1_rows
label_1_rows = label_1_rows.drop_duplicates(subset=[TGT_LANG_CODE+'_text'], keep='first')
label_1_rows['label'] = 2

resolved_dup_df = pd.concat([resolved_dup_df, label_1_rows], ignore_index=True, axis=0)
resolved_dup_df = resolved_dup_df.reset_index(drop=True)

print("resolved_dup_df:", resolved_dup_df.shape)
print("duplicates:", duplicates.shape)
print("duplicates_remaining:", duplicates_remaining.shape)

In [19]:
duplicates_remaining.shape

In [20]:
duplicates_remaining = duplicates_remaining.drop_duplicates(subset=[TGT_LANG_CODE+'_text'], keep='first')
duplicates_remaining.shape

In [21]:
duplicates_remaining

In [22]:
# duplicates_remaining sort by hin_text
duplicates_remaining = duplicates_remaining.sort_values(by=[TGT_LANG_CODE+'_text'])
duplicates_remaining = duplicates_remaining.reset_index(drop=True)
duplicates_remaining

In [23]:
now_str

In [24]:
duplicates_remaining.to_csv('../datasets/duplicates_remaining_'+TGT_LANG_CODE+'_'+ now_str+'.csv', index=False,  encoding="utf-8")

In [25]:
duplicates_remaining = duplicates_remaining.sort_values(by=['hin_text']).reset_index(drop=True)
duplicates_remaining


In [26]:
resolved_dup_df = resolved_dup_df.sort_values(by=['hin_text']).reset_index(drop=True)
resolved_dup_df

In [27]:
resolved_dup_df = pd.concat([resolved_dup_df, duplicates_remaining], ignore_index=True, axis=0)
resolved_dup_df = resolved_dup_df.reset_index(drop=True)
resolved_dup_df = resolved_dup_df.sort_values(by=['hin_text']).reset_index(drop=True)
resolved_dup_df


In [28]:
df_cleaned = pd.concat([df_cleaned, resolved_dup_df], ignore_index=True, axis=0)
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned = df_cleaned.sort_values(by=[TGT_LANG_CODE+'_text']).reset_index(drop=True)
df_cleaned = df_cleaned.drop_duplicates(subset=[TGT_LANG_CODE+'_text'], keep='first')
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned


In [30]:
now_str

In [29]:
df_cleaned.to_csv('../datasets/syn_data_3labelled_cleaned_'+TGT_LANG_CODE+'_'+ now_str+'.csv', index=False,  encoding="utf-8")

# Aim :: Same label 

In [1]:
import pandas as pd
import datetime as dt
import ast

In [2]:
# read 
df1 = pd.read_csv('../datasets/syn_data_3labelled_cleaned_hin_2025_04_11_18_28_56.csv', header=0)

df2 = pd.read_csv('../datasets/syn_data_3labelled_hin_2025-03-30_10-53-38.csv', header=0)

In [3]:
df1.head()

In [17]:
df1.shape

In [4]:
df2.head()

In [7]:
df2.shape

In [ ]:
# make a new df df3 that has same label and gemma_3_12b_label from df2
df_same_labels = df2.copy()
df_same_labels = df_same_labels[df_same_labels['label'] == df_same_labels['gemma3_12b_label']]
df_same_labels = df_same_labels.reset_index(drop=True)
df_same_labels.head()
df_same_labels.shape

In [ ]:
df_same_labels.head(20)

In [ ]:
df_same_labels['label'].value_counts()

In [ ]:
# percentag eof df2 is df3 by rows
percentage = (df_same_labels.shape[0] / df2.shape[0]) * 100
print("percentage of df2 is df3 by rows:", percentage)

In [13]:
df_diff_labels = df2.copy()
df_diff_labels = df_diff_labels[df_diff_labels['label'] != df_diff_labels['gemma3_12b_label']]
df_diff_labels = df_diff_labels.reset_index(drop=True)
# df_diff_labels.head()
df_diff_labels.shape

In [14]:
print("percentage of df2 is df_diff_labels by rows:", (df_diff_labels.shape[0] / df2.shape[0]) * 100)
df_diff_labels['label'].value_counts()

In [16]:
# group by label and gemma_3_12b_label and count the number of rows
df_diff_labels.groupby(['label', 'gemma3_12b_label']).size()

In [21]:
# is df1 a subset of df2 only by comparing the strip of column 'hin_text' 
df1['hin_text'] = df1['hin_text'].str.strip()
df2['hin_text'] = df2['hin_text'].str.strip()
# now compare the two dataframes find the overlapping rows of hin_text
df1_hin_text = df1['hin_text'].tolist()
df2_hin_text = df2['hin_text'].tolist()
df1_hin_text_set = set(df1_hin_text)
df2_hin_text_set = set(df2_hin_text)
overlapping_hin_text = df1_hin_text_set.intersection(df2_hin_text_set)
print("overlapping_hin_text:", overlapping_hin_text)



# Extract train.src_lang+train.tgt_lang for FT

In [313]:
for word in indic_sent_dict.keys():
    for tgt_lang in lang_script_list[:2]:
        print("Word: ", word, "TGT_LANG: ", tgt_lang)
        # sum =0
        for k in indic_sent_dict[word][tgt_lang].keys():
            # sum += len(indic_sent_dict[word][tgt_lang][k])
            print("    key: ", k, " len: ", len(indic_sent_dict[word][tgt_lang][k]) , "\n\n")
            for sent in indic_sent_dict[word][tgt_lang][k][:50]:
                print(word+','+tgt_lang+','+k+','+sent)
        # print("==Total: ", sum)

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd


### short pre-processing

In [138]:
# eng_ind_sents = pd.read_csv("/home/sofia/git_repos/Multilingual_project/notebooks/eng_indic_texts.csv", header=0, encoding="utf-8")
eng_ind_sents = pd.read_csv("/data/users/sofia/eng_indic_texts.csv", header=0, encoding="utf-8")
eng_ind_sents = eng_ind_sents.dropna(subset=['eng_text', 'tgt_lang_text'])
eng_ind_sents = eng_ind_sents.drop_duplicates( keep='first')
eng_ind_sents.head()
print("shape of eng_ind_sents:", eng_ind_sents.shape)

In [139]:
eng_ind_sents.columns

In [140]:
count =0
for word in ambiguous_words:
    for tgt_lang in lang_script_list:
      # if tgt_lang in ['hin_Deva', 'guj_Gujr', ]:
        # print("Word: ", word, "TGT_LANG: ", tgt_lang)
        count_list =[]
        for k in pir[word][tgt_lang].keys():
            # check if same word, lang, k for eng_indic_texts has value >=50
            t = eng_ind_sents[(eng_ind_sents['word'] == word) & (eng_ind_sents['tgt_lang'] == tgt_lang) & (eng_ind_sents['indic_term'] == k)]
            count_list.append(t.shape[0])
            if t.shape[0] ==0:
                count+=1
                print(f"Word: {word}, TGT_LANG: {tgt_lang}, Key: {k}")# does not have sufficient data.", t.shape[0])
            

        # print("Word: ", word, "TGT_LANG: ", tgt_lang, "Count List: ", count_list, "Total Count:", sum(count_list))
        # if np.std(count_list, ddof=1) >5:
        #     print(f"Word: {word}, TGT_LANG: {tgt_lang} has high variance in counts across keys. Standard Deviation: {np.std(count_list, ddof=1)}") 
        #     print("keys: "  , pir[word][tgt_lang].keys(), count_list)
        

        # print no.of lines to  delete  the extra lines apart from min (count_list) from each key also print respective key
        # min_count = min(count_list)
        # for k, c in zip(pir[word][tgt_lang].keys(), count_list):
        #     if c > min_count:
        #         print(f"Word: {word}, TGT_LANG: {tgt_lang}, Key: {k} has {c} lines. Need to delete {c - min_count} lines.")
            # else:
            #     print(f"Word: {word}, TGT_LANG: {tgt_lang}, Key: {k} has sufficient data with {c} lines.")

# print("Total count of insufficient data for any word, tgt_lang, key:", count)
count


In [141]:
from rapidfuzz import process, fuzz


In [142]:
# check for all rows in eng_indic_texts if word is present in eng_indic_texts['eng_text'] and
# indic_term is present in eng_indic_texts['tgt_lang_text']
# if not then print the row and delete the row
print("shape of eng_indic_texts before filtering:", eng_ind_sents.shape)
for index, row in eng_ind_sents.iterrows():
    # Check for NaN before using 'in'
    tgt_lang_text = row['tgt_lang_text']
    eng_text = row['eng_text']
    indic_term = row['indic_term']
    tgt_lang = row['tgt_lang']
    word = row['word']
    if pd.isnull(tgt_lang_text) or pd.isnull(indic_term) or fuzz.partial_ratio(indic_term, str(tgt_lang_text).lower()) < 80:
        print("indic term not found in tgt_lang_text:", indic_term, "in row:", tgt_lang_text)
        eng_ind_sents.drop(index, inplace=True)
        continue  # skip further checks if already dropped
    if pd.isnull(eng_text) or pd.isnull(word) or word not in str(eng_text).lower():
        print("--", f"{word},{tgt_lang},{indic_term},{tgt_lang_text},{eng_text}" )
        eng_ind_sents.drop(index, inplace=True)

    # if indic_term not in pir[word][tgt_lang].keys():
    if not process.extractOne(indic_term, pir[word][tgt_lang].keys(), score_cutoff=80):

        print("indic_term not found in pir[word][",tgt_lang,"].keys():", indic_term, "in row:", tgt_lang_text, "keys:", pir[word][tgt_lang].keys(), indic_term)
        eng_ind_sents.drop(index, inplace=True)

print("shape of eng_indic_texts after filtering:", eng_ind_sents.shape)
eng_ind_sents.to_csv('/data/users/sofia/eng_indic_texts.csv', index=False, encoding="utf-8")

## blaancing

In [143]:
from sklearn.utils import resample

# df_agg_resampled = pd.DataFrame()
# for lang in lang_script_list:

#     for word in eng_ind_sents[eng_ind_sents['tgt_lang']== lang]['indic_term'].unique():
#         print("Word: ", word)
#         # get the rows with word
#         lang_df = eng_ind_sents[eng_ind_sents['tgt_lang']== lang]
#         word_rows = lang_df[lang_df['indic_term'] == word]
#         print("shape of word_rows:", word_rows.shape)
#         word_rows = resample(word_rows, replace=True, n_samples=50, random_state=42)
#         print("shape of word_rows after resampling:", word_rows.shape)
#         # save the rows to a csv file
#         # word_rows.to_csv(f"eng_indic_{word}.csv", index=False, encoding="utf-8")
#         df_agg_resampled = pd.concat([df_agg_resampled, word_rows], ignore_index=True, axis=0)
# print("shape of df_agg_resampled:", df_agg_resampled.shape, df_agg_resampled.groupby(['indic_term', ]).size())


In [144]:
def balance_df(df, col, num):
    """
    Balance the dataframe by resampling the specified column to have a fixed number of samples.
    
    Parameters:
    df (DataFrame): The input dataframe.
    col (str): The column to balance.
    num (int): The number of samples to balance to.
    
    Returns:
    DataFrame: The balanced dataframe.
    """
    balanced_df = pd.DataFrame()
    # print("Before the balancing, shape of df:", df.shape, df.groupby([col]).size())
    for value in df[col].unique():
        subset = df[df[col] == value]
        if len(subset) < num:
            subset = resample(subset, replace=True, n_samples=num, random_state=42)
        else:
            subset = subset.sample(n=num, random_state=42)
        balanced_df = pd.concat([balanced_df, subset], ignore_index=True, axis=0)
        # print(f"After balancing for {value}, shape of balanced_df: {balanced_df.shape}, count: {balanced_df[col].value_counts()[value]}")
    # print("Final shape of balanced_df:", balanced_df.shape, balanced_df.groupby([col]).size())
    return balanced_df

In [145]:
# segragate the sentences into groups of tgt_lang and make train.{tgt_lang} text file of tgt_lang_text and
# train.eng_Latn for eng_text
tgt_langs = ['hin_Deva', 'guj_Gujr', 'mar_Deva', 'ory_Orya', 'ben_Beng', 'tam_Taml', 'pan_Guru', 'tel_Telu', 'mal_Mlym', 'kan_Knda']
for tgt_lang in tgt_langs:
    # make a new df with only the tgt_lang and eng_Latn columns
    df_tgt_lang = eng_ind_sents[ eng_ind_sents['tgt_lang'] == tgt_lang]#[['tgt_lang_text', 'eng_text']]
    # df_tgt_lang = df_agg_resampled[ df_agg_resampled['tgt_lang'] == tgt_lang]#[['tgt_lang_text', 'eng_text']]

    df_tgt_lang = df_tgt_lang.dropna()
    df_tgt_lang = df_tgt_lang.reset_index(drop=True)
    # split the df_tgt_lang into train and test sets using train_test_split
    if df_tgt_lang.empty:
        print("DataFrame is empty. Skipping train-test split.", tgt_lang    )
    else:
        # print("shape of df_tgt_lang:", df_tgt_lang.shape, "for tgt_lang:", tgt_lang)
        # print("Shape of word wise df_tgt_lang:", df_tgt_lang.groupby('indic_term').size(), df_tgt_lang.shape, "for tgt_lang:", tgt_lang)
        df_train, df_test = train_test_split(df_tgt_lang, test_size=0.2, random_state=42, shuffle=True, stratify=df_tgt_lang['indic_term'])
        df_train = balance_df(df_train, 'indic_term', 40)
        df_test = balance_df(df_test, 'indic_term', 10)

# assert that df_train when grouped by indic term, all groups have same no.of rows = 40
        # print("shape of df_train:", df_train.shape, "for tgt_lang:", tgt_lang)
        print("Shape of word wise df_train:", df_train.groupby('indic_term').size(), df_train.shape, "for tgt_lang:", tgt_lang)
        # print("shape of df_test:", df_test.shape, "for tgt_lang:", tgt_lang)    
        # assert df_train.groupby('indic_term').size().min() == 40, f"Not all groups in df_train have 40 rows for {tgt_lang}"
        # assert df_test.groupby('indic_term').size().min() == 10, f"Not all groups in df_test have 10 rows for {tgt_lang}"
        
        
        # # save the df_tgt_lang to a text file - /data/users/sofia/indictrans2_FT/en_ind_FT
        with open('/data/users/sofia/indictrans2_FT/en_ind_FT/train/eng_Latn-' + tgt_lang+ '/train.' + tgt_lang, 'w', encoding='utf-8') as f:
            for index, row in df_train.iterrows():
                f.write(row['tgt_lang_text'] + '\n')
        with open('/data/users/sofia/indictrans2_FT/en_ind_FT/train/eng_Latn-'+tgt_lang+'/train.eng_Latn', 'w', encoding='utf-8') as f:
            for index, row in df_train.iterrows():
                f.write(row['eng_text'] + '\n')

        with open('/data/users/sofia/indictrans2_FT/en_ind_FT/devtest/all/eng_Latn-' + tgt_lang+ '/dev.' + tgt_lang, 'w', encoding='utf-8') as f:
            for index, row in df_test.iterrows():
                f.write(row['tgt_lang_text'] + '\n')
        with open('/data/users/sofia/indictrans2_FT/en_ind_FT/devtest/all/eng_Latn-'+tgt_lang+'/dev.eng_Latn', 'w', encoding='utf-8') as f:
            for index, row in df_test.iterrows():
                f.write(row['eng_text'] + '\n')

## checking for more prompted data

In [ ]:
# eng_ind_sents = pd.read_csv("/home/sofia/git_repos/Multilingual_project/notebooks/eng_indic_texts.csv", header=0, encoding="utf-8")
# eng_ind_sents.head()

In [69]:
# categories = eng_indic_texts.groupby(['word', 'tgt_lang', 'indic_term']).size()
# categories

In [70]:
# import numpy as np

# Conclusion: